# **1. Perkenalan Dataset**


Tahap pertama, Anda harus mencari dan menggunakan dataset dengan ketentuan sebagai berikut:

1. **Sumber Dataset**:  
   Dataset dapat diperoleh dari berbagai sumber, seperti public repositories (*Kaggle*, *UCI ML Repository*, *Open Data*) atau data primer yang Anda kumpulkan sendiri.


# **2. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning atau deep learning.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE


# **3. Memuat Dataset**

Pada tahap ini, Anda perlu memuat dataset ke dalam notebook. Jika dataset dalam format CSV, Anda bisa menggunakan pustaka pandas untuk membacanya. Pastikan untuk mengecek beberapa baris awal dataset untuk memahami strukturnya dan memastikan data telah dimuat dengan benar.

Jika dataset berada di Google Drive, pastikan Anda menghubungkan Google Drive ke Colab terlebih dahulu. Setelah dataset berhasil dimuat, langkah berikutnya adalah memeriksa kesesuaian data dan siap untuk dianalisis lebih lanjut.

Jika dataset berupa unstructured data, silakan sesuaikan dengan format seperti kelas Machine Learning Pengembangan atau Machine Learning Terapan

In [ ]:
import os

# Cara Sederhana: Menggunakan relative path
# Tanda '..' artinya naik satu level ke folder di atasnya
try:
    df = pd.read_csv('../bank-additional-full_raw.csv', sep=';')
except FileNotFoundError:
    # Opsi cadangan jika dijalankan dari folder root (misal di VS Code tertentu)
    df = pd.read_csv('Eksperimen_SML_Moch-Arief-Kresnanda/bank-additional-full_raw.csv', sep=';')

print("Dataset berhasil dimuat:")
display(df.head())

# **4. Exploratory Data Analysis (EDA)**

Pada tahap ini, Anda akan melakukan **Exploratory Data Analysis (EDA)** untuk memahami karakteristik dataset.

Tujuan dari EDA adalah untuk memperoleh wawasan awal yang mendalam mengenai data dan menentukan langkah selanjutnya dalam analisis atau pemodelan.

In [ ]:
# dataset info
print("Dataset Info:")
df.info()

# missing values
print("\nMissing Values:")
print(df.isnull().sum())

# duplicates
print(f"\nDuplicate Rows: {df.duplicated().sum()}")

# Statistical summary
display(df.describe())

# Target variable distribution
plt.figure(figsize=(6, 4))
sns.countplot(x='y', data=df)
plt.title('Distribution of Target Variable (y)')
plt.show()

# Check categorical variables
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    print(f"\nUnique values in {col}:")
    print(df[col].value_counts())

# **5. Data Preprocessing**

Pada tahap ini, data preprocessing adalah langkah penting untuk memastikan kualitas data sebelum digunakan dalam model machine learning.

Jika Anda menggunakan data teks, data mentah sering kali mengandung nilai kosong, duplikasi, atau rentang nilai yang tidak konsisten, yang dapat memengaruhi kinerja model. Oleh karena itu, proses ini bertujuan untuk membersihkan dan mempersiapkan data agar analisis berjalan optimal.

Berikut adalah tahapan-tahapan yang bisa dilakukan, tetapi **tidak terbatas** pada:
1. Menghapus atau Menangani Data Kosong (Missing Values)
2. Menghapus Data Duplikat
3. Normalisasi atau Standarisasi Fitur
4. Deteksi dan Penanganan Outlier
5. Encoding Data Kategorikal
6. Binning (Pengelompokan Data)

Cukup sesuaikan dengan karakteristik data yang kamu gunakan yah. Khususnya ketika kami menggunakan data tidak terstruktur.

In [ ]:
# 1. Drop 'duration' column
df_clean = df.drop(columns=['duration'])

# 2. Handle duplicates
df_clean = df_clean.drop_duplicates()
print(f"Shape after dropping duplicates: {df_clean.shape}")

# --- TAMBAHAN 1: Feature Engineering pdays ---
# Ubah pdays 999 menjadi kategori baru atau biner
df_clean['previously_contacted'] = np.where(df_clean['pdays'] == 999, 0, 1)
df_clean = df_clean.drop(columns=['pdays']) 

# --- TAMBAHAN 2: Capping Outliers pada 'campaign' ---
# Membatasi nilai campaign di persentil 99 agar tidak ada nilai ekstrem
upper_limit = df_clean['campaign'].quantile(0.99)
df_clean['campaign'] = np.where(df_clean['campaign'] > upper_limit, upper_limit, df_clean['campaign'])

# 3. Separate features and target
X = df_clean.drop(columns=['y'])
y = df_clean['y']

# 4. Encode target variable
le = LabelEncoder()
y = le.fit_transform(y)
print("Target classes:", le.classes_)

# 5. Identify columns types
# --- REVISI STRATEGI ENCODING (HYBRID) ---

# A. Ordinal (Punya urutan atau Biner)
ordinal_cols = ['education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week']

# Definisikan urutan untuk masing-masing
education_order = ['illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school', 'professional.course', 'university.degree', 'unknown']
default_order = ['no', 'yes', 'unknown']
housing_order = ['no', 'yes', 'unknown']
loan_order    = ['no', 'yes', 'unknown']
contact_order = ['telephone', 'cellular']
# REVISI: Tambahkan jan dan feb agar urutan lengkap secara logis
month_order   = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
day_order     = ['mon', 'tue', 'wed', 'thu', 'fri']

all_ordinal_categories = [
    education_order, default_order, housing_order, loan_order, 
    contact_order, month_order, day_order
]

# B. Nominal (Murni Kategori, tanpa urutan) -> Tetap OneHot
categorical_cols = ['job', 'marital', 'poutcome']

# C. Numerical
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
# Pastikan kolom yang sudah masuk ordinal/categorical tidak masuk numerical
numerical_cols = [col for col in numerical_cols if col not in ordinal_cols and col not in categorical_cols]

print(f"Numerical columns: {list(numerical_cols)}")
print(f"Nominal (OneHot) columns: {list(categorical_cols)}")
print(f"Ordinal columns: {list(ordinal_cols)}")

# 6. Preprocessing pipeline
# Numerical
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('yeo_johnson', PowerTransformer(method='yeo-johnson')),
    ('scaler', StandardScaler())
])

# Categorical (Nominal): OneHot
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), 
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Categorical (Ordinal): Ordinal Encoding
ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), 
    ('ordinal', OrdinalEncoder(categories=all_ordinal_categories, handle_unknown='use_encoded_value', unknown_value=-1))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat_nominal', categorical_transformer, categorical_cols),
        ('cat_ordinal', ordinal_transformer, ordinal_cols)
    ])

# 7. Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 8. Apply preprocessing
# Fit on training data, transform on both
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# --- TAMBAHAN 3: Handle Imbalance dengan SMOTE ---
print("Before SMOTE counts:", np.bincount(y_train))
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_processed, y_train)
print("After SMOTE counts:", np.bincount(y_train_resampled))

# Update variabel training data ke hasil resampling
X_train_processed = X_train_resampled
y_train = y_train_resampled

# Get feature names
try:
    cat_nominal_names = preprocessor.named_transformers_['cat_nominal']['onehot'].get_feature_names_out(categorical_cols)
    feature_names = list(numerical_cols) + list(cat_nominal_names) + list(ordinal_cols)
    
    # Convert back to dataframe for verification
    X_train_df = pd.DataFrame(X_train_processed, columns=feature_names)
    print("Processed Train Data Shape:", X_train_df.shape)
    display(X_train_df.head())
except Exception as e:
    print(f"Could not create DataFrame: {e}")
    print("Processed Train Data Shape:", X_train_processed.shape)

print("Preprocessing complete.")